# 02 RAG | 01 Chat Completion | 07 Chat Completion Image

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIResponseClient

The `AzureOpenAIClient` from the Microsoft.Extensions.AI NuGet package offers an abstraction which allows using models from multiple providers.

In this notebook the `OpenAIResponseClient` is used.

The multi modal capabilities of the GPT-4o LLM deployment are shown by providing the following photo realistic image together with the user message 'In which style is the provided image? Is it Cartoon or Photorealistic?' the LLM:

![PhotoRealisticImage](../../Assets/Image/SuperBowl_Photorealistic.png)

The image was created using the [04_CreateImageCompletion notebook](./04_CreateImageCompletion.ipynb) and an instance of dall-e-3.

In [3]:
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"
#r "nuget: DotNetEnv, 3.1.1"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI;
using OpenAI.Responses;
using DotNetEnv;
using System;
using System.IO;
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "CHATCOMPLETION_DEPLOYMENTNAME not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);
OpenAIResponseClient responseClient = azureOpenAIClient.GetOpenAIResponseClient(chatCompletionDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"OpenAIResponseClient created...");

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.4 DotNetEnv, 3.1.1

AzureOpenAI Client created...
OpenAIResponseClient created...


## Step 1: Create Chat Messages

This step demonstrates a basic multimodal interaction using the `OpenAIResponseClient` object in the Azure OpenAI Responses API. The assistant receives both a text prompt and an image input, allowing it to provide context-aware analysis based on visual content.

The `ResponseCreationOptions` object gives more fine granular control how the model should process the provided prompt.

The `Instructions` property on the `ResponseCreationOptions` object is where you define the assistant's behavior, tone, or domain-specific guidance when you're working with Azure's Responses API.

Instructions: `"You describe the style of images!"`

Prompt: `"In which style is the image? Is it cartoon or photorealistic?"`

In [26]:

// Prepare response creation options
ResponseCreationOptions options = new ResponseCreationOptions
{
    Instructions = "You describe the style of images!",
    TruncationMode = ResponseTruncationMode.Auto
};

// Load image bytes from file system
string imagePath = "../../Assets/Image/SuperBowl_Sketch.png";
BinaryData imageBytes = BinaryData.FromBytes(await File.ReadAllBytesAsync(imagePath));


// Create a multimodal user message with text and image
var multimodalMessage = ResponseItem.CreateUserMessageItem(
    new ResponseContentPart[]
    {
        ResponseContentPart.CreateInputTextPart("In which style is the image? Is it cartoon or photorealistic?"),
        ResponseContentPart.CreateInputImagePart(imageBytes, "image/png", ResponseImageDetailLevel.Low)
    }
);

// Assemble all messages (single multimodal message in this case)
List<ResponseItem> messages = new List<ResponseItem> { multimodalMessage };

Console.WriteLine($"Multi modal message created!");


Multi modal message created!


## Step 2: Call Model

The method `CreateResponseAsync()`of the `OpenAIResponseClient` object can be used to provide the data to the model and retrieve the model response.


In [27]:
// Send request to model
OpenAIResponse response = await responseClient.CreateResponseAsync(messages, options);

// Output results
Console.WriteLine($"Model response: \n {response.GetOutputText()}"); 

Model response: 
 The image is in a cartoon style. It features bold lines, simplified forms, and vibrant colors, particularly in the confetti, giving it a playful and illustrative feel.
